<a href="https://colab.research.google.com/github/aim56009/AI-MA_project/blob/main/Chew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import torchvision.transforms.functional as TF 

from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import glob
import os

import random
import click
import sklearn
import sklearn.model_selection
from collections import defaultdict
from sklearn.metrics import accuracy_score
from pathlib import Path
import sys
from torch import optim
from torch.optim import lr_scheduler
from functools import partial
import matplotlib.pyplot as plt

In [2]:
!git clone https://github.com/aim56009/AI-MA_project.git

Cloning into 'AI-MA_project'...
remote: Enumerating objects: 2310, done.
remote: Counting objects: 100% (2310/2310), done.
remote: Compressing objects: 100% (193/193), done.
remote: Total 2310 (delta 2189), reused 2174 (delta 2115), pack-reused 0
Receiving objects: 100% (2310/2310), 4.50 MiB | 5.51 MiB/s, done.
Resolving deltas: 100% (2189/2189), done.
Checking out files: 100% (2219/2219), done.


# Chew


In [3]:
!pip3 install partitura
import partitura
import numpy as np
import os
import pickle
from sklearn.metrics import accuracy_score

     |████████████████████████████████| 218 kB 5.1 MB/s 
     |████████████████████████████████| 273 kB 55.6 MB/s 
     |████████████████████████████████| 103 kB 49.9 MB/s 
     |████████████████████████████████| 51 kB 4.5 MB/s 
     |████████████████████████████████| 181 kB 58.3 MB/s 


In [ ]:
def evaluation_chew():

    history = defaultdict(list)
    path = "AI-MA_project/chorales_converted"

    ### load the chorales in xml format ###
    for filename in os.listdir(path):
        if not filename.endswith('.xml'): continue
        fullname = os.path.join(path, filename)
        

        part = partitura.load_musicxml(fullname)
            

        ### apply chews method ### 
        chew_sep = partitura.musicanalysis.estimate_voices(part, monophonic_voices=True)


        ### seperate the results -> e.g. pos_zero = all positions where chew prediction says voice 0 ####
        pos_zero = np.where(chew_sep==1)
        pos_one = np.where(chew_sep==2)
        pos_two = np.where(chew_sep==3)
        pos_three = np.where(chew_sep==4)


        ### create notearray object that contain only the corresponding voice ###
        note_array_zero = partitura.utils.ensure_notearray(part)[pos_zero]
        note_array_one = partitura.utils.ensure_notearray(part)[pos_one]
        note_array_two = partitura.utils.ensure_notearray(part)[pos_two]
        note_array_three = partitura.utils.ensure_notearray(part)[pos_three]


        ### create pr representation of all voices ###
        pr_zero = partitura.utils.compute_pianoroll(note_array_zero,time_unit = "beat",time_div = 12)
        pr_zero = pr_zero.toarray()

        pr_one = partitura.utils.compute_pianoroll(note_array_one,time_unit = "beat",time_div = 12)
        pr_one = pr_one.toarray()

        pr_two = partitura.utils.compute_pianoroll(note_array_two,time_unit = "beat",time_div = 12)
        pr_two = pr_two.toarray()

        pr_three = partitura.utils.compute_pianoroll(note_array_three,time_unit = "beat",time_div = 12)
        pr_three = pr_three.toarray()


        ### get original data (as labels) ###
        part_zero = part[0]
        part_one = part[1]
        part_two = part[2]
        part_three = part[3]


        ### compute pianoroll representation of original data for computing acc ###
        pr_zero_ori = partitura.utils.compute_pianoroll(part_zero, time_unit = "beat",time_div = 12)
        v0 = pr_zero_ori.toarray()

        pr_one_ori = partitura.utils.compute_pianoroll(part_one, time_unit = "beat",time_div = 12)
        v1 = pr_one_ori.toarray()

        pr_two_ori = partitura.utils.compute_pianoroll(part_two, time_unit = "beat",time_div = 12)
        v2 = pr_two_ori.toarray()

        pr_three_ori = partitura.utils.compute_pianoroll(part_three, time_unit = "beat",time_div = 12)
        v3 = pr_three_ori.toarray()

        pr_ori = partitura.utils.compute_pianoroll(part, time_unit = "beat",time_div = 12)
        v_all = pr_ori.toarray()

        if pr_zero.shape == pr_one.shape == pr_two.shape == pr_three.shape  :
            ### compute accuracy ###
            acc_v0 = accuracy_score(torch.tensor(pr_zero), v0)
            acc_v1 = accuracy_score(torch.tensor(pr_one), v1)
            acc_v2 = accuracy_score(torch.tensor(pr_two),v2)
            acc_v3 = accuracy_score(torch.tensor(pr_three), v3)

            history["accuracy_v0"].append(acc_v0)
            history["accuracy_v1"].append(acc_v1)
            history["accuracy_v2"].append(acc_v2)
            history["accuracy_v3"].append(acc_v3)
            
        else: print("pr shape does not match for:", filename)
        
    return history

In [ ]:
hist_acc =  evaluation_chew()

pr shape does not match for: chor245.xml
pr shape does not match for: chor264.xml
pr shape does not match for: chor286.xml
pr shape does not match for: chor056.xml
pr shape does not match for: chor283.xml
pr shape does not match for: chor074.xml
pr shape does not match for: chor043.xml
pr shape does not match for: chor252.xml
pr shape does not match for: chor133.xml
pr shape does not match for: chor095.xml


In [ ]:
total_len = len(hist_acc["accuracy_v0"])
total_len

360

In [ ]:
accuracy_v0 = np.sum(hist_acc["accuracy_v0"])/total_len
accuracy_v1 = np.sum(hist_acc["accuracy_v1"])/total_len
accuracy_v2 = np.sum(hist_acc["accuracy_v2"])/total_len
accuracy_v3 = np.sum(hist_acc["accuracy_v3"])/total_len

print("accuracy of v0-v3:",accuracy_v0,accuracy_v1,accuracy_v2,accuracy_v3)

accuracy of v0-v3: 0.9882161458333333 0.9801215277777777 0.9844618055555555 0.9959201388888889


# HMM approach

In [ ]:
fullname = "chor001.xml"

#part = partitura.load_score_midi(fullname,part_voice_assign_mode=4,estimate_voice_info=False)
part = partitura.load_musicxml(fullname)
partitura.save_score_midi(part,fullname[:8]+"mid")

In [26]:
part = partitura.load_score_midi("chor238.mid",part_voice_assign_mode=4,estimate_voice_info=False)
part

In [27]:
pr = partitura.utils.compute_pianoroll(part, time_unit = "beat",time_div = 12)
v0 = pr.toarray()
v0.shape

(128, 480)

In [28]:
voice_info = partitura.utils.note_array_from_part(part) 
voice_info["voice"]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
partitura.utils.ensure_notearray(part)

In [7]:
%cd AI-MA_project
!pwd

/content/AI-MA_project
/content/AI-MA_project


In [20]:
filename[:8]+"mid"

'chor155.mid'

In [8]:
#path = "AI-MA_project/chorales_converted"
path = "chorales_converted"


for filename in os.listdir(path):
    if not filename.endswith('.xml'): continue
    fullname = os.path.join(path, filename)
    #print(fullname)
    part = partitura.load_musicxml(fullname)
    #print(filename[:8]+"mid")
    try:
      partitura.save_score_midi(part,out= "midi_test/"+filename[:8]+"mid",part_voice_assign_mode=4)
    except:
      print(filename[:8]+"mid")


chor155.mid
chor162.mid
chor210.mid
chor015.mid
chor187.mid
chor244.mid
chor241.mid
chor237.mid
chor113.mid
chor122.mid
chor206.mid
chor087.mid
chor180.mid
chor302.mid
chor160.mid
chor199.mid
chor357.mid
chor184.mid
chor110.mid
chor284.mid
chor154.mid
chor288.mid
chor205.mid
chor127.mid
chor369.mid
chor034.mid
chor025.mid
chor166.mid
chor171.mid
chor203.mid
chor232.mid
chor119.mid
chor019.mid
chor174.mid
chor262.mid
chor231.mid
chor168.mid
chor008.mid
chor186.mid
chor082.mid
chor197.mid
chor133.mid
chor003.mid
chor070.mid
chor049.mid
chor218.mid
chor325.mid
chor185.mid


In [ ]:
from google.colab import files

files.download('midi_test/')

In [14]:
!pwd

/content/AI-MA_project


In [20]:
path = "midi_test/"
for filename in os.listdir(path):
    print(filename)
    files.download(path + filename)


chor271.mid


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

chor195.mid


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

chor062.mid


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>